In [ ]:
import os
pwd = os.getcwd()

blaspp_source = "/home/weslleyp/storage/blaspp"
lapackpp_source = "/home/weslleyp/storage/lapackpp"
tlapack_source = "/home/weslleyp/storage/tlapack"

tlapack_DIR = pwd+"/tlapack"
tlapackMKL_DIR = pwd+"/tlapack_mkl"
blaspp_DIR = pwd+"/blaspp"
lapackpp_DIR = pwd+"/lapackpp"
eigen3_DIR = "/home/weslleyp/storage/eigen/eigen_master/share/eigen3/cmake"

from datetime import datetime
from decimal import Decimal
seed = datetime.now().timestamp()
seed = seed - int(seed)
seed = 1000*seed
seed = int(seed)

seed = 604
matrix_type = 54

import random
random.seed(seed)

import numpy as np
import matplotlib.pyplot as plt
import matplotlib
from IPython.display import display, Math

In [ ]:
seed

In [ ]:
# System:
!uname -a

In [ ]:
# Machine:
!lscpu

In [ ]:
# Eigen version:
!cat "$eigen3_DIR/Eigen3ConfigVersion.cmake" | grep "set(PACKAGE_VERSION \""

In [ ]:
# Eigen version:
!cat /usr/local/lib/cmake/mdspan/mdspanConfigVersion.cmake | grep "set(PACKAGE_VERSION \""

In [ ]:
# MKL version:
!which mkl_link_tool

In [ ]:
!ls $eigen3_DIR

In [ ]:
#Build without MKL

# Install <T>LAPACK
!cmake -B "$tlapack_DIR" -G Ninja -D CMAKE_BUILD_TYPE=Release -D BUILD_EXAMPLES=OFF -D BUILD_TESTING=OFF -D TLAPACK_NDEBUG=ON -D CMAKE_INSTALL_PREFIX="$tlapack_DIR" -D CMAKE_INSTALL_MESSAGE="LAZY" "$tlapack_source"
!cmake --build "$tlapack_DIR" --target install

# Build
!cmake -B build -G Ninja -D CMAKE_BUILD_TYPE=Release -D Eigen3_DIR="$eigen3_DIR" -D CMAKE_PREFIX_PATH="."
!cmake --build build

In [ ]:
!./build/performance_eigen 100 1 1 {matrix_type} {seed}

In [ ]:
!./build/performance_tlapack 100 1 1 {matrix_type} {seed}

In [ ]:
#Build with MKL

# Install BLAS++
!cmake -B "$blaspp_DIR" -G Ninja -D CMAKE_BUILD_TYPE=Release -D build_tests=OFF -D CMAKE_INSTALL_PREFIX="$blaspp_DIR" -D CMAKE_INSTALL_MESSAGE="LAZY" "$blaspp_source"
!cmake --build "$blaspp_DIR" --target install

# Install LAPACK++
!cmake -B "$lapackpp_DIR" -G Ninja -D CMAKE_BUILD_TYPE=Release -D build_tests=OFF -D CMAKE_INSTALL_PREFIX="$lapackpp_DIR" -D CMAKE_INSTALL_MESSAGE="LAZY" -D blaspp_DIR="$blaspp_DIR" "$lapackpp_source"
!cmake --build "$lapackpp_DIR" --target install

# Install <T>LAPACK
!cmake -B "$tlapackMKL_DIR" -G Ninja -D CMAKE_BUILD_TYPE=Release -D BUILD_EXAMPLES=OFF -D BUILD_TESTING=OFF -D TLAPACK_NDEBUG=ON -D CMAKE_INSTALL_PREFIX="$tlapackMKL_DIR" -D CMAKE_INSTALL_MESSAGE="LAZY" -D TLAPACK_USE_LAPACKPP=ON -D blaspp_DIR="$blaspp_DIR" -D lapackpp_DIR="$lapackpp_DIR" "$tlapack_source"
!cmake --build "$tlapackMKL_DIR" --target install

# Build
!cmake -B build_mkl -G Ninja -D CMAKE_BUILD_TYPE=Release -D tlapack_DIR="$tlapackMKL_DIR" -D blaspp_DIR="$blaspp_DIR" -D lapackpp_DIR="$lapackpp_DIR" -D USE_MKL=ON -D Eigen3_DIR="$eigen3_DIR"
!cmake --build build_mkl

In [ ]:
!./build_mkl/performance_eigen 500 1 1 {matrix_type} {seed}

In [ ]:
!./build_mkl/performance_eigen_blasMKL 500 1 1 {matrix_type} {seed}

In [ ]:
!./build_mkl/performance_tlapack 500 1 1 {matrix_type} {seed}

In [ ]:
nSizes = [10, 25, 50, 100, 200, 400, 800, 1600]
N = len(nSizes)

datatypes = ["float","double"]
NT = len(datatypes)

nRuns = 3

executable = [
    "build/performance_tlapack",
    "build/performance_eigen"
]
methods = [
    r"<T>LAPACK - C++",
    r"Eigen3 - C++"
]
M = len(executable)

In [ ]:
data = np.ones([M,N,NT], dtype=np.float64) * 60 * 60 * 24

for s in range(M):
    for i in range(N):
        n = nSizes[i]
        for j in range(nRuns):
            expr = executable[s]
            output = !$expr {n} 0 0 {matrix_type} {seed} | grep time
            for k in range(NT):
                data[s,i,k] = np.minimum( float(output[k].split()[2]), data[s,i,k] )

In [ ]:
data

In [ ]:
markers = ['x-','*-','+-']
plt.rcParams['font.size'] = 12

for datatype in range(NT):
    print(datatypes[datatype])

    fig1, ax1 = plt.subplots()

    for m in range(M):
        plt.plot(nSizes,data[m,:,datatype],markers[m%3],label=methods[m])

    ax1.set_xscale("log")
    ax1.set_yscale("log")
    ax1.set_xticks(nSizes)
    ax1.get_xaxis().set_major_formatter(matplotlib.ticker.ScalarFormatter())

    plt.xlabel("n")
    plt.ylabel("time (s)")
    plt.legend()

    plt.tight_layout()
    plt.savefig("curvesWithNoMKL_"+datatypes[datatype]+".pdf")
    plt.show()

In [ ]:
nSizes = [10, 25, 50, 100, 200, 400, 800, 1600]
N = len(nSizes)

datatypes = ["float","double"]
NT = len(datatypes)

nRuns = 3

executable = [
    "build_mkl/performance_tlapack",
    "build_mkl/performance_eigen",
    "build_mkl/performance_eigen_blasMKL"
]
methods = [
    r"<T>LAPACK using MKL BLAS",
    r"MKL gees (Eigen3 wrapper)",
    r"Eigen3 using MKL BLAS"
]
M = len(executable)

In [ ]:
data_mkl = np.ones([M,N,NT], dtype=np.float64) * 60 * 60 * 24

for s in range(M):
    for i in range(N):
        n = nSizes[i]
        for j in range(nRuns):
            expr = executable[s]
            output = !$expr {n} 0 0 {matrix_type} {seed} | grep time
            for k in range(NT):
                data_mkl[s,i,k] = np.minimum( float(output[k].split()[2]), data_mkl[s,i,k] )

In [ ]:
markers = ['x-','*-','+-']
plt.rcParams['font.size'] = 12

for datatype in range(NT):
    print(datatypes[datatype])

    fig1, ax1 = plt.subplots()

    for m in range(M):
        plt.plot(nSizes,data_mkl[m,:,datatype],markers[m%3],label=methods[m])

    ax1.set_xscale("log")
    ax1.set_yscale("log")
    ax1.set_xticks(nSizes)
    ax1.get_xaxis().set_major_formatter(matplotlib.ticker.ScalarFormatter())

    plt.xlabel("n")
    plt.ylabel("time (s)")
    plt.legend()

    plt.tight_layout()
    plt.savefig("curvesWithMKL_"+datatypes[datatype]+".pdf")
    plt.show()

In [ ]:
markers = ['x-','*-','+-']
plt.rcParams['font.size'] = 12

methods = [
    r"Eigen3 - C++",
    r"<T>LAPACK - C++",
    r"Eigen3 using MKL BLAS",
    r"<T>LAPACK using MKL BLAS",
    r"MKL gees (Eigen3 wrapper)"
]

for datatype in range(NT):
    print(datatypes[datatype])

    fig1, ax1 = plt.subplots()
    
    plt.plot(
        nSizes,
        np.divide( data[1,:,datatype], data[1,:,datatype] ),
        '--',
        label = methods[0])
    plt.plot(
        nSizes,
        np.divide( data[1,:,datatype], data[0,:,datatype] ),
        markers[1],
        label = methods[1])
    plt.plot(
        nSizes,
        np.divide( data[1,:,datatype], data_mkl[2,:,datatype] ),
        markers[0],
        label = methods[2])
    plt.plot(
        nSizes,
        np.divide( data[1,:,datatype], data_mkl[0,:,datatype] ),
        markers[1],
        label = methods[3])
    plt.plot(
        nSizes,
        np.divide( data[1,:,datatype], data_mkl[1,:,datatype] ),
        markers[0],
        label = methods[4])

    ax1.set_xscale("log")
    # ax1.set_yscale("log")
    ax1.set_xticks(nSizes)
    ax1.get_xaxis().set_major_formatter(matplotlib.ticker.ScalarFormatter())

    plt.xlabel("n")
    plt.ylabel("Speedup")
    plt.legend()

    plt.tight_layout()
    plt.savefig("speedup_"+datatypes[datatype]+".pdf")
    plt.show()

# Backward stability analysis

In [ ]:
nSizes = [10, 25, 50, 100, 200, 400, 800, 1600]
N = len(nSizes)

datatypes = ["float","double"]
NT = len(datatypes)

executable = [
    "build/performance_tlapack",
    "build/performance_eigen"
]
M = len(executable)

methods = [
    r"<T>LAPACK - C++",
    r"Eigen3 - C++"
]

errors = [
    [
        r"$\|Z^H Z - I\|/\|I\|$",
        r"$\|Z Z^H - I\|/\|I\|$", 
        r"$\|Z T Z^H - A\|/\|A\|$" #,
        # r"$\|Q^H Q - I\|/\|I\|$",
        # r"$\|Q Q^H - I\|/\|I\|$"
    ],
    [
        r"$\|Z^H Z - I\|/\|I\|$",
        r"$\|Z Z^H - I\|/\|I\|$",
        r"$\|Z T Z^H - A\|/\|A\|$"
        # r"$\|V \Lambda - A V\|/(\|A\| \|V\|)$",
        # r"$\|V \Lambda V^{-1} - A\|/\|A\|$"
    ]
]
NE = 6

In [ ]:
data_bwError = np.ones([M,N,NT,NE], dtype=np.float64)

for s in range(M):
    for i in range(N):
        n = nSizes[i]
        expr = executable[s]
        output = !$expr {n} 0 1 {matrix_type} {seed} | grep "||"
        # print(output)
        for k in range(NT):
            for j in range( len(errors[s]) ):
                try:
                    data_bwError[s,i,k,j] = float(output[k*len(errors[s])+j].split()[-1])
                except Exception:
                    data_bwError[s,i,k,j] = float("nan")

In [ ]:
markers = ['x-','*-','+-']
plt.rcParams['font.size'] = 12

for datatype in range(NT):
    for m in range(M):
        fig1, ax1 = plt.subplots()
        
        print(datatypes[datatype])
        print(methods[m])

        for i in range( len(errors[m]) ):
            plt.plot(nSizes,data_bwError[m,:,datatype,i],markers[i % 3],label=errors[m][i])

        ax1.set_xscale("log")
        # ax1.set_yscale("log")
        ax1.set_xticks(nSizes)
        ax1.get_xaxis().set_major_formatter(matplotlib.ticker.ScalarFormatter())

        plt.xlabel("n")
        plt.ylabel("Relative error")
        plt.legend()

        plt.tight_layout()
        plt.savefig("error_"+datatypes[datatype]+"_"+methods[m]+".pdf")
        plt.show()

# Testing with mdspan

In [ ]:
# Build without MKL
!cmake -B build_mdspan -G Ninja -D CMAKE_BUILD_TYPE=Release -D CMAKE_PREFIX_PATH="." -D USE_MDSPAN_DATA=ON
!cmake --build build_mdspan --target performance_tlapack

# Build with MKL
!cmake -B build_mkl_mdspan -G Ninja -D CMAKE_BUILD_TYPE=Release -D tlapack_DIR="$tlapackMKL_DIR" -D blaspp_DIR="$blaspp_DIR" -D lapackpp_DIR="$lapackpp_DIR" -D USE_MKL=ON -D USE_MDSPAN_DATA=ON
!cmake --build build_mkl_mdspan --target performance_tlapack

In [ ]:
nSizes = [10, 25, 50, 100, 200, 400, 800, 1600]
N = len(nSizes)

datatypes = ["float","double"]
NT = len(datatypes)

nRuns = 3

In [ ]:
data_mdspan = np.ones([N,NT], dtype=np.float64) * 60 * 60 * 24

executable = "build_mdspan/performance_tlapack"
for i in range(N):
    n = nSizes[i]
    for j in range(nRuns):
        output = !$executable {n} 0 0 {matrix_type} {seed} | grep time
        for k in range(NT):
            data_mdspan[i,k] = np.minimum( float(output[k].split()[2]), data_mdspan[i,k] )

In [ ]:
data_mkl_mdspan = np.ones([N,NT], dtype=np.float64) * 60 * 60 * 24

executable = "build_mkl_mdspan/performance_tlapack"
for i in range(N):
    n = nSizes[i]
    for j in range(nRuns):
        output = !$executable {n} 0 0 {matrix_type} {seed} | grep time
        for k in range(NT):
            data_mkl_mdspan[i,k] = np.minimum( float(output[k].split()[2]), data_mkl_mdspan[i,k] )

In [ ]:
data_mdspan

In [ ]:
data_mkl_mdspan

In [ ]:
markers = ['x-','*-','+-']
plt.rcParams['font.size'] = 12

for datatype in range(NT):
    print(datatypes[datatype])

    fig1, ax1 = plt.subplots()

    plt.plot(nSizes,data[0,:,datatype],markers[0],label=r"Eigen::Matrix")
    plt.plot(nSizes,data_mdspan[:,datatype],markers[1],label=r"kokkos::mdspan")

    plt.plot(nSizes,data_mkl[0,:,datatype],markers[0],label=r"Eigen::Matrix (MKL BLAS)")
    plt.plot(nSizes,data_mkl_mdspan[:,datatype],markers[1],label=r"kokkos::mdspan (MKL BLAS)")

    ax1.set_xscale("log")
    ax1.set_yscale("log")
    ax1.set_xticks(nSizes)
    ax1.get_xaxis().set_major_formatter(matplotlib.ticker.ScalarFormatter())

    plt.xlabel("n")
    plt.ylabel("time (s)")
    plt.legend()

    plt.tight_layout()
    plt.savefig("curves_mdspan_"+datatypes[datatype]+".pdf")
    plt.show()

    fig1, ax1 = plt.subplots()

    plt.plot(nSizes,np.divide(data[0,:,datatype],data_mdspan[:,datatype]),markers[1],label=r"C++ only")
    plt.plot(nSizes,np.divide(data_mkl[0,:,datatype],data_mkl_mdspan[:,datatype]),markers[1],label=r"Using MKL BLAS")

    ax1.set_xscale("log")
    # ax1.set_yscale("log")
    ax1.set_xticks(nSizes)
    ax1.get_xaxis().set_major_formatter(matplotlib.ticker.ScalarFormatter())

    plt.xlabel("n")
    plt.ylabel("Speedup")
    plt.legend()

    plt.tight_layout()
    plt.savefig("speedup_mdspan_"+datatypes[datatype]+".pdf")
    plt.show()